In [1]:
!pip install -q pandas      # To run queries on data
!pip install -q openai      # To call OpenAI's API
!pip install -q tiktoken    # To count tokens in text
!pip install -q langchain   # To orchestrate function calling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import userdata
import os

# Let's make sure LLMPROXY_JWT is defined.
assert userdata.get("LLMPROXY_JWT")

os.environ['OPENAI_API_KEY'] = f"{userdata.get('LLMPROXY_JWT')}:function-calling-workshop"
os.environ['OPENAI_API_BASE'] = 'https://llmfoundry.straive.com/v1/'

In [4]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'Call right function.'),
        ('human', '{input}'),
        ('human', 'Always use right format'),
    ]
)

In [6]:
import pandas as pd
from typing import Optional
bank = pd.read_csv('/content/drive/MyDrive/GenAI/Function_Call/bank-full.csv', sep=";", encoding='cp1252')

In [7]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [92]:
def get_person_without_debt(count: Optional[int] = 5):
  """
  Filters a DataFrame to return rows where the 'loan', 'housing', and 'creditcard' columns are all 'no'.

  Args:
    data (pd.DataFrame): The input DataFrame.

  Returns:
    pd.DataFrame: A DataFrame containing rows with no debt.
  """
  no_debt_filter = (bank['loan'] == 'no') & (bank['housing'] == 'no') & (bank['default'] == 'no')
  return bank[no_debt_filter].head(count)[['age', 'job', 'marital', 'education', "balance", "campaign", 'default','housing', 'loan', 'y']]

In [93]:
get_person_without_debt(3)

,age,job,marital,education,balance,campaign,default,housing,loan,y
4,33,unknown,single,unknown,1,1,no,no,no,no
37,53,technician,married,secondary,-3,1,no,no,no,no
48,55,technician,divorced,secondary,0,1,no,no,no,no


In [37]:
def bank_balance_test(count: Optional[int] = 5, min_bal: Optional[int] = 0, max_bal: Optional[int] = 99999999):
    """
    In this fuction we can able to get balance range to user
    input values.

    Args:
        count (Optional[int]): The number of accounts to return.
        min_bal (Optional[int]): The minimum balance for the 'range' query.
        max_bal (Optional[int]): The maximum balance for the 'range' query.

    """

    result = bank[bank['balance'] >= min_bal] if min_bal else bank
    result = result[result['balance'] <= max_bal] if max_bal else result
    return result.sort_values('balance', ascending=False).head(count)[['age', 'job', 'marital', 'education', "balance", "campaign", 'default','housing', 'loan', 'y']]

In [38]:
bank_balance_test(3,800,1200)

,age,job,marital,education,balance,campaign,default,housing,loan,y
21176,50,management,single,tertiary,1200,3,no,no,no,yes
11091,40,admin.,single,secondary,1200,8,no,yes,no,no
27493,40,admin.,single,secondary,1200,2,no,yes,no,no


In [39]:
def have_term_insurance(count: Optional[int] = 5, insurance:Optional[str] = 'no'):
    """In this func we can able to get the
    individual person already taken insurance or not.

    Args:
        count: number of apartments to return
        insurance: Having yes or no
    """

    result = bank[bank['y'] == insurance] if insurance else bank

    return result.head(count)[['age', 'job', 'marital', 'education', "balance", "campaign",'default','housing', 'loan', 'y']]

In [40]:
have_term_insurance(3, 'yes')

,age,job,marital,education,balance,campaign,default,housing,loan,y
83,59,admin.,married,secondary,2343,1,no,yes,no,yes
86,56,admin.,married,secondary,45,1,no,no,no,yes
87,41,technician,married,secondary,1270,1,no,yes,no,yes


In [41]:
def campaign_client(count: Optional[int] = 5, contact_client: Optional[int] = 1):
    """In this func we can able to get the
    number of contact made to cleint in the campaign.

    Args:
        count: number of apartments to return
        contact_client: min to max attment contact to client
    """

    result = bank[bank['campaign'] >= contact_client] if contact_client else bank

    return result.head(count)[['age', 'job', 'marital', 'education', "balance", "campaign", 'default','housing', 'loan', 'y']]

In [42]:
campaign_client(4,3)

,age,job,marital,education,balance,campaign,default,housing,loan,y
103,59,blue-collar,married,primary,179,3,no,yes,no,no
122,34,admin.,married,unknown,3,3,no,yes,no,no
158,52,blue-collar,married,secondary,782,3,no,yes,no,no
163,40,blue-collar,married,secondary,1225,5,no,yes,no,no


In [43]:
def house_loan(count: Optional[int] = 5, loan:Optional[str] = 'no'):
    """In this func we can able to get the
    individual person having house loan or not.

    Args:
        count: number of apartments to return
        house_loan: Having yes or no
    """

    result = bank[bank['housing'] == loan] if loan else bank

    return result.head(count)[['age', 'job', 'marital', 'education', "balance", "campaign",'default','housing', 'loan', 'y']]

In [44]:
house_loan(3,'yes')

,age,job,marital,education,balance,campaign,default,housing,loan,y
0,58,management,married,tertiary,2143,1,no,yes,no,no
1,44,technician,single,secondary,29,1,no,yes,no,no
2,33,entrepreneur,married,secondary,2,1,no,yes,yes,no


In [94]:
from langchain.chains.openai_functions import create_openai_fn_chain

#user_query = input("Enter your query: ")
functions = [
    bank_balance_test,
    have_term_insurance,
    house_loan,
    get_person_without_debt,
    campaign_client
]
chain = create_openai_fn_chain(functions, llm, prompt,  verbose=True)

In [103]:
query = 'show me count of 3 person without debt?'
result = chain.generate([{'input': query}])
result

LLMResult(generations=[[ChatGeneration(generation_info={'finish_reason': 'function_call', 'logprobs': None}, message=AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{}', 'name': 'get_person_without_debt'}}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 266, 'total_tokens': 279}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae', 'finish_reason': 'function_call', 'logprobs': None}, id='run-33e57653-d048-4412-8788-fff59c01edd7-0'))]], llm_output={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 266, 'total_tokens': 279}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae'}, run=[RunInfo(run_id=UUID('33e57653-d048-4412-8788-fff59c01edd7'))])

In [98]:
query = 'show me the contact of client more than 2 in campaign?'
result = chain.generate([{'input': query}])
result

LLMResult(generations=[[ChatGeneration(generation_info={'finish_reason': 'function_call', 'logprobs': None}, message=AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"contact_client":["2",""]}', 'name': 'campaign_client'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 269, 'total_tokens': 286}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae', 'finish_reason': 'function_call', 'logprobs': None}, id='run-66e1b36a-2ad6-40e6-9574-e3d104e7b573-0'))]], llm_output={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 269, 'total_tokens': 286}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae'}, run=[RunInfo(run_id=UUID('66e1b36a-2ad6-40e6-9574-e3d104e7b573'))])

In [109]:
import json
from datetime import datetime, timezone

# Cost of gpt-3.5-turbo as of 10 Oct 2023
cents = {
    'prompt_tokens': 0.0015 / 1000 * 100,
    'completion_tokens': 0.002 / 1000 * 100,
}
fn_map = {fn.__name__: fn for fn in functions}

def answer(query: str, today: str = datetime.now(timezone.utc).strftime('%Y-%m-%d')) -> dict:
    response = chain.generate([{'input': query, 'today': today}])
    # Calculate cost of query in cents
    cost = sum(cents[token] * response.llm_output['token_usage'][token] for token in cents)
    result = {'data': None, 'cost': cost, 'query': query, 'response': response}
    # Get the function name and arguments
    if len(response.generations) == 0 or len(response.generations[0]) == 0:
        return result
    function = response.generations[0][0].message.additional_kwargs.get('function_call', None)
    if function is None or function.get('name', None) not in fn_map:
        return result
    # Call the function and return the result
    kwargs = json.loads(function['arguments'])
    result['data'] = fn_map[function['name']](**kwargs)
    return result

In [110]:
answer1 = answer('what is highest bank balance account?')
answer1['data']

,age,job,marital,education,balance,campaign,default,housing,loan,y
39989,51,management,single,tertiary,102127,1,no,no,no,no
26227,59,management,married,tertiary,98417,5,no,no,no,no
43393,84,retired,married,secondary,81204,1,no,no,no,yes
42558,84,retired,married,secondary,81204,1,no,no,no,yes
41693,60,retired,married,primary,71188,1,no,no,no,no


In [99]:
answer2 = answer('show me the contact of client more than 2 in campaign?')
answer2['data']

,age,job,marital,education,balance,campaign,default,housing,loan,y
28,46,management,single,secondary,-246,2,no,yes,no,no
41,50,management,married,secondary,49,2,no,yes,no,no
47,44,technician,married,secondary,0,2,no,yes,no,no
68,55,technician,married,secondary,1205,2,no,yes,no,no
85,51,blue-collar,married,primary,173,2,no,yes,no,no


In [66]:
answer3 = answer('show me person who holding term insurance?')
answer3['data']

,age,job,marital,education,balance,campaign,default,housing,loan,y
83,59,admin.,married,secondary,2343,1,no,yes,no,yes
86,56,admin.,married,secondary,45,1,no,no,no,yes
87,41,technician,married,secondary,1270,1,no,yes,no,yes
129,55,services,married,secondary,2476,1,no,yes,no,yes
168,54,admin.,married,tertiary,184,2,no,no,no,yes


In [67]:
answer4 = answer('show me the person without having house loan?')
answer4['data']

,age,job,marital,education,balance,campaign,default,housing,loan,y
4,33,unknown,single,unknown,1,1,no,no,no,no
30,57,technician,married,secondary,839,1,no,no,yes,no
37,53,technician,married,secondary,-3,1,no,no,no,no
48,55,technician,divorced,secondary,0,1,no,no,no,no
55,38,entrepreneur,single,tertiary,243,1,no,no,yes,no


In [111]:
answer5 = answer('show me count of 3 person without debt?')
answer5['data']

,age,job,marital,education,balance,campaign,default,housing,loan,y
4,33,unknown,single,unknown,1,1,no,no,no,no
37,53,technician,married,secondary,-3,1,no,no,no,no
48,55,technician,divorced,secondary,0,1,no,no,no,no
66,60,retired,married,tertiary,100,1,no,no,no,no
77,55,services,divorced,secondary,91,1,no,no,no,no


In [112]:
answer1['cost'], answer2['cost'], answer3['cost'], answer4['cost'], answer5['cost']

(0.041650000000000006, 0.0189, 0.037250000000000005, 0.0362, 0.0425)

## Reduce the cost

The cost is driven by the number of functions we pass to OpenAI. With more functions, the cost per query increases.

Instead, let's find the 2 most similar queries (based on vector embeddings) and pass them to OpenAI. This reduces the cost to about 0.03 cents.

In [113]:
import numpy as np
from langchain.storage.file_system import LocalFileStore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from typing import List

file_store = LocalFileStore('.embeddings/')
base = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(base, file_store, namespace=base.model)

def classify(docs: List[str], topics: List[str]):
    """Return the similarity between each doc and topic"""
    doc_embed = np.array(cached_embeddings.embed_documents(docs))
    topic_embed = np.array(cached_embeddings.embed_documents(topics))
    return np.dot(doc_embed, topic_embed.T)

Now modify `answer()` to pick the top 2 queries and pass them to OpenAI.

In [115]:
#from langchain.chains.openai_functions.base import convert_python_function_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function

def answer(query: str, today: str = datetime.now(timezone.utc).strftime('%Y-%m-%d'), top=2, min_similarity=0.85) -> dict:
    # Convert the functions into a JSON schema
    questions = []
    for name, fn in fn_map.items():
        schema = convert_to_openai_function(fn)
        for desc in schema['description'].split('\n'):
            questions.append({'q': desc.strip(), 'fn': fn})
    # Find the similarity of each function to the query
    similar = classify([query], [question['q'] for question in questions])
    for index, similarity in enumerate(similar[0]):
        questions[index]['similarity'] = similarity
    # Create chain from the top similar questions with min_similarity
    questions = sorted(questions, key=lambda q: q['similarity'], reverse=True)
    top_functions = [q['fn'] for q in questions if q['similarity'] >= min_similarity][:top]
    chain = create_openai_fn_chain(top_functions, llm, prompt, verbose=True)
    chain = create_openai_fn_chain(top_functions, llm, prompt, verbose=True)

    # Then run the chain. This is the same as before
    response = chain.generate([{'input': query, 'today': today}])
    # Calculate cost of query in cents
    cost = sum(cents[token] * response.llm_output['token_usage'][token] for token in cents)
    result = {'data': None, 'cost': cost, 'query': query, 'response': response, 'functions': top_functions}
    # Get the function name and arguments
    if len(response.generations) == 0 or len(response.generations[0]) == 0:
        return result
    function = response.generations[0][0].message.additional_kwargs.get('function_call', None)
    if function is None or function.get('name', None) not in fn_map:
        return result
    # Call the function and return the result
    kwargs = json.loads(function['arguments'])
    result['data'] = fn_map[function['name']](**kwargs)
    return result

In [125]:
#answer1a = answer('what is highest bank balance account?')
answer2a = answer('show me the contact of client more than 2 in campaign?')
#answer3a = answer('show me person who holding term insurance?')
answer4a = answer('show me the person without having house loan?')
#answer5a = answer('show me count of 3 person without debt?')

In [127]:
answer2a['data'].T

,28,41,47,68,85
age,46,50,44,55,51
job,management,management,technician,technician,blue-collar
marital,single,married,married,married,married
education,secondary,secondary,secondary,secondary,primary
balance,-246,49,0,1205,173
campaign,2,2,2,2,2
default,no,no,no,no,no
housing,yes,yes,yes,yes,yes
loan,no,no,no,no,no
y,no,no,no,no,no


In [128]:
answer4a['data'].T

,4
age,33
job,unknown
marital,single
education,unknown
balance,1
campaign,1
default,no
housing,no
loan,no
y,no


In [129]:
answer2a['cost'], answer4a['cost']

(0.0189, 0.01525)

In [130]:
answer2['data'].equals(answer2a['data']), answer4['data'].equals(answer4a['data'])

(True, False)

In [131]:
[
  answer1['data'].equals(answer2a['data']),
  answer2['data'].equals(answer4a['data'])
]

[False, False]